In [ ]:
# This notebook is about correlation analysis of signature similarity and logp of inhibitors in given cell line & gene
# Signature similarity: spearman rank correlation of signature induced by shRNA and signature induced by inhibitor

# Gene signature: gene expression data from L1000 (GSE92742) -- Level 3 & level 5
# Inhibiors source of gene of interest(GOI): using TTD, DrugBank & IC50, Ki<= 10μM

In [1]:
import numpy as np
import pandas as pd
import os
from cmapPy.pandasGEXpress.parse_gctx import parse
from scipy.spatial.distance import cosine
from utils import get_mol
from rdkit.Chem import Descriptors
from scipy.stats import mannwhitneyu, spearmanr
import pickle as pkl
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = 'Arial'
import matplotlib as mpl
import seaborn as sns
from utils import tanimoto_similarity_smiles
from rdkit.Chem.Draw import MolsToGridImage

### Inhibitors extraction
+ Part1: TTD & DrugBank: inhitibor definition `['antagonist', 'inhibitor', 'substrate|inhibitor', 
               'antagonist|inhibitor', 'negative modulator','blocker',
               'blocker (channel blocker)', 'downregulator','inhibitor|downregulator',
               'inhibitor|binder', 'inhibitor (gating inhibitor)', 'antagonist|inhibitor|ligand',
               'antagonist|binder', 'inhibitor|blocker', 'antagonist|downregulator', 'antagonist|substrate',
               'suppressor','inhibitory allosteric modulator', 'antagonist|ligand']`
+ Part2: `IC50, Ki < 5μM`, affinity data from `TTD, BindingDB & Chembl`

+ This data extraction procedure is done in notebook: **`../DrugPartition2/240318-DataExtract-L1000-Pert-target-affinity.ipynb`**

+ data saved: 
+ > target2inhibitors&pert_id: `./data/L1000/GSE92742_trt_cp_union_target2inhibitors_maps_5uM.pkl`
+ > affinity maps `{smi: {(uniprot, measure): affinity, (uniprot2, measure2): affinity2}}`: `./data/L1000/GSE92742_trt_cp_affinity_maps.pkl`

### Correlation of signature similarity & logP
+ Gene knock down: shRNA, trt_sh.cgs

In [ ]:
# perturbagen_info_df = pd.read_csv('/home/oyj/data/L1000_Cmap/GSE92742_Broad_LINCS_pert_info.txt', sep='\t', header=0)
# Gene LoF (shRNAs for LoF)
# shrna_perturbagen_info_df = perturbagen_info_df[perturbagen_info_df['pert_type'] == 'trt_sh.cgs']
# shrna_perturbagen_info_df = shrna_perturbagen_info_df.iloc[:, :3]
# print(shrna_perturbagen_info_df.shape)  # (4345, 3)

# target mapping
# target_lof_maps_df = pd.read_csv('./data/tmp/L1000_target_inhibited_mapping_2024_03_21.tsv', sep='\t')
# gene2uniprot = {}
# for uniprot, gene_names in target_lof_maps_df[['Entry', 'Gene Names']].values:
#     for gene in gene_names.split(' '):
#         gene2uniprot[gene] = uniprot

# shrna_perturbagen_info_df['uniprot'] = shrna_perturbagen_info_df['pert_iname'].apply(
#     lambda item: gene2uniprot[item] if item in gene2uniprot else None)
# keep_shrna_perturbagen_info_df = shrna_perturbagen_info_df.dropna().reset_index(drop=True)

# print(keep_shrna_perturbagen_info_df.shape)  # (965, 4)

In [2]:
# Using GSE92742: LINCS Phase I L1000 dataset
# compute similarity between signature induced by shRNA.cgs and signature induced by inhibitor
gene_info_df = pd.read_csv('/home/oyj/data/L1000_Cmap/GSE92742_Broad_LINCS_gene_info.txt', sep='\t', header=0, dtype=str)
# Total genes
# print(gene_info_df.shape)
all_gene_ids = gene_info_df['pr_gene_id'].tolist()
# Landmark genes
landmark_gene_ids = gene_info_df[gene_info_df.pr_is_lm == '1']['pr_gene_id'].tolist()
print(len(all_gene_ids), len(landmark_gene_ids))  # 12328 978
# gene_info_df.head(2)

# Knock-down genes with inhibitors information
keep_shrna_perturbagen_info_df = pd.read_csv('./data/L1000/GSE92742_trt_sh.cgs_info.csv')
# keep_shrna_perturbagen_info_df.head(2)

# target to inhibitors mapping dict
# using union: DrugBank + TTD + Affinity (IC50, Ki) defined
# with open('./data/L1000/GSE92742_trt_cp_union_target2inhibitors_maps.pkl', 'rb') as file:
#     target_lof2smi_pert_id = pkl.load(file=file)
with open('./data/L1000/GSE92742_trt_cp_union_target2inhibitors_maps_5uM.pkl', 'rb') as file:
    target_lof2smi_pert_id = pkl.load(file=file)
# affinity data
with open('./data/L1000/GSE92742_trt_cp_affinity_maps.pkl', 'rb') as file:
    affinity_maps = pkl.load(file=file)

gene2uniprot = {gene: uniprot for gene, uniprot in keep_shrna_perturbagen_info_df[['pert_iname', 'uniprot']].values}
uniprot2gene = {uniprot: gene for gene, uniprot in keep_shrna_perturbagen_info_df[['pert_iname', 'uniprot']].values}

# signature information
signature_info_df = pd.read_csv('/home/oyj/data/L1000_Cmap/GSE92742_Broad_LINCS_sig_info.txt', 
                                sep='\t', header=0, low_memory=False)
trt_cp_signature_info_df = signature_info_df[signature_info_df['pert_type'] == 'trt_cp']
trt_sh_cgs_signature_info_df = signature_info_df[signature_info_df['pert_type'] == 'trt_sh.cgs']

12328 978


In [3]:
# Somes bugs in Chembl database: remove them manually
ESR1_agonists = [
    'C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CCC2=O',  # estrone
    'CC12CCC3c4ccc(O)cc4CCC3C1CCC2O',  # Estra-1,3,5(10)-triene-3,17-diol
    'C#C[C@]1(O)CC[C@H]2[C@@H]3CCc4cc(O)ccc4[C@H]3CC[C@@]21C',  # Ethynylestradiol
    'C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@@H](O)[C@@H]2O',  # estriol
    'C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C@@H]2O'  # estradiol
]
print(len(target_lof2smi_pert_id['P03372']))
tmp = set()
for item in target_lof2smi_pert_id['P03372']:
    if item[0] not in ESR1_agonists:
        tmp.add(item)
target_lof2smi_pert_id['P03372'] = tmp
print(len(target_lof2smi_pert_id['P03372']))

38
33


In [4]:
genes = ['ESR1', 'EGFR', 'KDR', 'AR', 'NR3C1', 'TOP2A', 'TOP2B', 'PGR']

gene2uniprot['PGR'] = 'P06401'
uniprot_ids = [gene2uniprot[gene] for gene in genes]
for uni in uniprot_ids:
    print(len(target_lof2smi_pert_id[uni]))

33
99
79
28
41
21
14
20


In [4]:
def get_inhibitors_signature_similarity(uniprot_id, cell_line = 'PC3',
                                        target2smi_dict=target_lof2smi_pert_id, 
                                        gene_ids = landmark_gene_ids,
                                        trt_cp_time = '24 h'):
    """
    For a protein (defined by `uniprot_id`), compute the similarity of shRNA LoF signature and
    inhibitor-induced signature (averaged).
    inputs:
        uniprot_id
        cell_line: 'PC3', 'MCF7'..  # For trt_sh.cgs & trt_cp
        target2smi_dict: uniprot mapping to (smi, pert_id) tuple
        gene_ids: using landmark genes or total genes (default: using landmark genes as other genes were infered)
        trt_cp_time: # For trt_cp: 24 h or 6 h
    returns:
        dict of similarity {smiles: sim}
        spearman correlation of logp & signature similarity
    """
    # set experiment conditions
    keep_trt_cp_signature_info_df = trt_cp_signature_info_df[(trt_cp_signature_info_df['cell_id'] == cell_line) &
                                                            (trt_cp_signature_info_df['pert_idose'] == '10 µM') &
                                                            (trt_cp_signature_info_df['pert_itime'] == trt_cp_time)]
    keep_trt_sh_cgs_signature_info_df = trt_sh_cgs_signature_info_df[trt_sh_cgs_signature_info_df['cell_id'] == cell_line]
    
    # trt_sh.cgs
    gene_lof_pert_id = keep_shrna_perturbagen_info_df[
        keep_shrna_perturbagen_info_df.uniprot == uniprot_id]['pert_id'].tolist()[0]
    gene_lof_sig_id = keep_trt_sh_cgs_signature_info_df[
        keep_trt_sh_cgs_signature_info_df.pert_id == gene_lof_pert_id]['sig_id']
    if gene_lof_sig_id.empty:
        return [], [], {}, ([], [])
    # level5 data: moderated z-scores (aggregate replicates, 
    # level4 --  calculated z-socre compared to control --that is differential expression values)
    gene_lof_sig_data = parse('/home/oyj/data/L1000_Cmap/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx', 
                            make_multiindex=True, rid=gene_ids, cid=gene_lof_sig_id.tolist())
    gene_lof_sig_values = gene_lof_sig_data.multi_index_df.values

    # trt_cp
    smi_and_pert_ids = list(target2smi_dict[uniprot_id])
    keep_ihb_smi2sig_id = {}
    for ihb_smi, ihb_pert_id in smi_and_pert_ids:
        if '|' not in ihb_pert_id:
            ihb_sig_id = keep_trt_cp_signature_info_df[keep_trt_cp_signature_info_df.pert_id == ihb_pert_id]['sig_id']
        else:
            ihb_sig_id = keep_trt_cp_signature_info_df[keep_trt_cp_signature_info_df.pert_id.isin(ihb_pert_id.split('|'))]['sig_id']
        if not ihb_sig_id.empty:
            keep_ihb_smi2sig_id[ihb_smi] = ihb_sig_id.tolist()
    
    print('Inhibitors number:', len(keep_ihb_smi2sig_id))
    if not keep_ihb_smi2sig_id:
        return [], [], {}, ([], [])
    sim_dict = {}
    for ihb_smi, ihb_sig_id in keep_ihb_smi2sig_id.items():
        ihb_sig_data = parse('/home/oyj/data/L1000_Cmap/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx', 
                             make_multiindex=True, rid=gene_ids, cid=ihb_sig_id)
        ihb_lof_sig_values = ihb_sig_data.multi_index_df.values
        sim = spearmanr(np.mean(gene_lof_sig_values, axis=1), np.mean(ihb_lof_sig_values, axis=1)).statistic
        sim_dict[ihb_smi] = sim

    # spearman correlation of logp & sim
    logps = [Descriptors.MolLogP(get_mol(smi)) for smi in sim_dict.keys()]
    logp_sim_spr = spearmanr(logps, list(sim_dict.values()))
    print(logp_sim_spr)

    return sim_dict, (logps, list(sim_dict.values()))

In [5]:
def get_inhibitors_N(uniprot_id, target2smi_dict=target_lof2smi_pert_id, 
                     cell_line = 'PC3', trt_cp_time = '24 h'):
    # set experiment conditions
    keep_trt_cp_signature_info_df = trt_cp_signature_info_df[(trt_cp_signature_info_df['cell_id'] == cell_line) &
                                                            (trt_cp_signature_info_df['pert_idose'] == '10 µM') &
                                                            (trt_cp_signature_info_df['pert_itime'] == trt_cp_time)]
    keep_trt_sh_cgs_signature_info_df = trt_sh_cgs_signature_info_df[trt_sh_cgs_signature_info_df['cell_id'] == cell_line]
    
    # trt_sh.cgs
    gene_lof_pert_id = keep_shrna_perturbagen_info_df[
        keep_shrna_perturbagen_info_df.uniprot == uniprot_id]['pert_id']
    if gene_lof_pert_id.empty:
        return 0
    gene_lof_sig_id = keep_trt_sh_cgs_signature_info_df[
        keep_trt_sh_cgs_signature_info_df.pert_id == gene_lof_pert_id.tolist()[0]]['sig_id']
    if gene_lof_sig_id.empty:
        return 0

    # trt_cp
    smi_and_pert_ids = list(target2smi_dict[uniprot_id])
    keep_ihb_smi2sig_id = {}
    for ihb_smi, ihb_pert_id in smi_and_pert_ids:
        if '|' not in ihb_pert_id:
            ihb_sig_id = keep_trt_cp_signature_info_df[keep_trt_cp_signature_info_df.pert_id == ihb_pert_id]['sig_id']
        else:
            ihb_sig_id = keep_trt_cp_signature_info_df[keep_trt_cp_signature_info_df.pert_id.isin(ihb_pert_id.split('|'))]['sig_id']
        if not ihb_sig_id.empty:
            keep_ihb_smi2sig_id[ihb_smi] = ihb_sig_id.tolist()

    return len(keep_ihb_smi2sig_id)

In [6]:
genes = ['ESR1', 'EGFR', 'KDR', 'AR', 'NR3C1', 'TOP2A', 'TOP2B', 'PGR']

gene2uniprot['PGR'] = 'P06401'
uniprot_ids = [gene2uniprot[gene] for gene in genes]
cell_lines = ['MCF7', 'HCC515','A549','VCAP', 'PC3', 'A375', 'HT29']  # 'HA1E' 为normal 去除
data_points_N = np.zeros(shape=(len(genes), len(cell_lines)), dtype=np.int64)
for i in range(len(genes)):
    for j in range(len(cell_lines)):
        data_points_N[i, j] = get_inhibitors_N(uniprot_id=uniprot_ids[i], cell_line=cell_lines[j],
                                               target2smi_dict=target_lof2smi_pert_id)

In [8]:
data_points_N

array([[28, 20, 25, 25, 27, 11, 11],
       [94, 49, 81, 79, 84, 29, 27],
       [75, 32, 59, 56, 61, 20, 18],
       [24, 16, 22, 23, 24,  5,  5],
       [39, 19, 31, 39, 39,  5,  5],
       [20,  3, 14, 10, 21,  3,  3],
       [13,  0,  8,  7, 14,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]])

In [6]:
# genes = ['ESR1', 'EGFR', 'KDR', 'AR', 'NR3C1', 'TOP2A', 'TOP2B', 'PGR']
genes = ['ESR1', 'EGFR', 'KDR', 'AR']

# gene2uniprot['PGR'] = 'P06401'
uniprot_ids = [gene2uniprot[gene] for gene in genes]
cell_lines = ['MCF7', 'HCC515','A549','VCAP', 'PC3', 'A375', 'HT29']  # 'HA1E' 为normal 去除
data_points_N = np.zeros(shape=(len(genes), len(cell_lines)), dtype=np.int64)
for i in range(len(genes)):
    for j in range(len(cell_lines)):
        data_points_N[i, j] = get_inhibitors_N(uniprot_id=uniprot_ids[i], cell_line=cell_lines[j],
                                               target2smi_dict=target_lof2smi_pert_id)

In [7]:
# for gene expression: level 3
inst_info = pd.read_csv('/home/oyj/data/L1000_Cmap/GSE92742_Broad_LINCS_inst_info.txt', sep='\t', low_memory=False)

def get_cell_line_gene_expr(gene_name, cell_line):
    """
    Get gene (given by uniprot_id) expression level of certain cell_line;
    """
    gene_id = gene_info_df[gene_info_df.pr_gene_symbol == gene_name].pr_gene_id.tolist()
    if len(gene_id) == 0:
        print('Unmapped uniprot id.')
        return None
    if len(gene_id) > 1:
        print('One gene name mapping to more than one gene ids.')
    tmp = inst_info[(inst_info.cell_id == cell_line) & (inst_info.pert_type == 'ctl_untrt')]
    cids = tmp['inst_id'].tolist()

    gene_exprs = parse('/home/oyj/data/L1000_Cmap/GSE92742_Broad_LINCS_Level3_INF_ctl_untrt_n22072x12328.gctx', make_multiindex=True, 
                       rid=gene_id, cid=cids)
    return gene_exprs.multi_index_df.values.flatten().tolist()

In [10]:
gene_expr_mat = np.zeros(shape=(len(genes), len(cell_lines)), dtype=np.int64)
for i in range(len(genes)):
    for j in range(len(cell_lines)):
        gene_expr_mat[i, j] = np.median(get_cell_line_gene_expr(gene_name=genes[i], cell_line=cell_lines[j]))